## MAIN CODE

In [1]:
import os
import csv
import shutil
from PIL import Image

def convert_to_png(input_image, output_image):
    """
    Convert an image to PNG format while removing any EXIF metadata.
    """
    try:
        with Image.open(input_image) as img:
            img.info.pop("exif", None)  # Remove EXIF metadata
            img.save(output_image, format="PNG")
        print(f"Converted: {input_image} -> {output_image}")
    except Exception as e:
        print(f"Error processing {input_image}: {e}")

def traverse_and_convert(input_folder, output_folder, avif_folder, avif_csv):
    """
    Convert images to PNG, move AVIF files to a separate folder, and generate CSV.
    """
    avif_file_paths = []

    for root, dirs, files in os.walk(input_folder):
        # Skip output and avif folders
        dirs[:] = [d for d in dirs if os.path.abspath(os.path.join(root, d)) not in 
                   [os.path.abspath(output_folder), os.path.abspath(avif_folder)]]

        for file in files:
            input_file_path = os.path.join(root, file)
            relative_path = os.path.relpath(root, input_folder)
            output_subfolder = os.path.join(output_folder, relative_path)
            os.makedirs(output_subfolder, exist_ok=True)

            # Handle AVIF files (Move them to avif_folder)
            if file.lower().endswith(".avif"):
                avif_output_path = os.path.join(avif_folder, file)
                if os.path.abspath(input_file_path) != os.path.abspath(avif_output_path):
                    shutil.copy(input_file_path, avif_output_path)  # Copy AVIF file
                    print(f"Copied AVIF: {input_file_path} -> {avif_output_path}")
                else:
                    print(f"Skipped copying (same file): {input_file_path}")
                avif_file_paths.append([relative_path, file])

            # Convert other formats to PNG
            elif file.lower().endswith((".jpg", ".jpeg", ".webp", ".png")):
                output_file_name = os.path.splitext(file)[0] + ".png"
                output_file_path = os.path.join(output_subfolder, output_file_name)
                convert_to_png(input_file_path, output_file_path)

    # Save AVIF file paths to CSV
    with open(avif_csv, mode="w", newline="", encoding="utf-8") as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(["Relative Path", "Original AVIF Filename"])
        writer.writerows(avif_file_paths)

    print(f"CSV file with AVIF locations saved at: {avif_csv}")


# Example Usage

base_directory = r"C:\Users\keval\Desktop\Jersey\Jersey Images\New Season\Manchester City Black 2025-26"

input_directory = base_directory
output_directory = os.path.join(base_directory, "Output")
avif_folder = os.path.join(base_directory, "avif_to_convert")
avif_csv_path = os.path.join(base_directory, "avif_files.csv")

# Ensure necessary directories exist
os.makedirs(output_directory, exist_ok=True)
os.makedirs(avif_folder, exist_ok=True)

# Start conversion process
traverse_and_convert(input_directory, output_directory, avif_folder, avif_csv_path)


Converted: C:\Users\keval\Desktop\Jersey\Jersey Images\New Season\Manchester City Black 2025-26\image1.png -> C:\Users\keval\Desktop\Jersey\Jersey Images\New Season\Manchester City Black 2025-26\Output\.\image1.png
Converted: C:\Users\keval\Desktop\Jersey\Jersey Images\New Season\Manchester City Black 2025-26\image2.png -> C:\Users\keval\Desktop\Jersey\Jersey Images\New Season\Manchester City Black 2025-26\Output\.\image2.png
Converted: C:\Users\keval\Desktop\Jersey\Jersey Images\New Season\Manchester City Black 2025-26\shopping (1).webp -> C:\Users\keval\Desktop\Jersey\Jersey Images\New Season\Manchester City Black 2025-26\Output\.\shopping (1).png
Converted: C:\Users\keval\Desktop\Jersey\Jersey Images\New Season\Manchester City Black 2025-26\🖤 The Manchester City 25-26 Away Kit beautifully blends the club’s rich heritage with modern pe (1).jpg -> C:\Users\keval\Desktop\Jersey\Jersey Images\New Season\Manchester City Black 2025-26\Output\.\🖤 The Manchester City 25-26 Away Kit beautifu